In [37]:
import pandas as pd

In [47]:
df = pd.read_csv("Business_Financials_Data.csv")

In [48]:
# Step 1: Preview the first few rows and the column names
print(df.head())
print("Columns:", df.columns.tolist())

   Business ID Business State  Total Long-term Debt  Total Equity  \
0     41872013       Kentucky              16889000      18046000   
1     76232013           Iowa               6252000      18293621   
2    160992013          Texas              19200000     177858000   
3    197452013       Delaware             117592000     278773000   
4    241042013       Illinois               4408000      52064000   

   Debt to Equity  Total Liabilities  Total Revenue  Profit Margin  
0        0.935886           25986000      136753000       0.023663  
1        0.341758           14474000       34226553       0.265015  
2        0.107951           72787000      384196000       0.130413  
3        0.421820          558749000      444306000       0.196768  
4        0.084665           19898000      121541000       0.168305  
Columns: ['Business ID', 'Business State', 'Total Long-term Debt', 'Total Equity', 'Debt to Equity', 'Total Liabilities', 'Total Revenue', 'Profit Margin']


In [55]:
# Step 2: Check if required columns are in the dataset
required_col = ["Business ID",
                "Business State", 
                "Total Long-term Debt",
                "Total Equity",
                "Debt to Equity", 
                "Total Liabilities", 
                "Total Revenue", 
                "Profit Margin"]

for col in required_col:
    if col not in df.columns:
        print(col, "is missing!")

In [56]:
# Step 3: Remove rows with missing values
df_clean = df.dropna()
print(f"Rows after cleaning: {len(df_clean)}")

Rows after cleaning: 150


In [57]:
# Step 4: Identify duplicate rows
dups = df_clean[df_clean.duplicated()]
if not dups.empty: 
    print("Duplicates detected. Please review the following rows:")
    print(dups)
    print("No rows have been deleted yet. Please remove them manually if appropriate.")
    # Stop the script so the analyst can review 
    raise SystemExit("Execution stopped for review of duplicate rows.")
else: 
    print("No duplicate rows found.")

No duplicate rows found.


In [69]:
# Step 5a: Safeguard for division by 0
dti_val = []
for _, row in df_clean.iterrows():
    if row["Total Revenue"] == 0:
        dti_val.append(0)
    else:
        dti_val.append(row["Total Long-term Debt"] / row["Total Revenue"]) 

# Step 5b - Create new dataframe for Debt-to-Income ratios
dti_df = pd.DataFrame()
dti_df["Business ID"] = df_clean["Business ID"]
dti_df["DebtToIncome"] = dti_val

print(dti_df.head())


   Business ID  DebtToIncome
0     41872013      0.123500
1     76232013      0.182665
2    160992013      0.049974
3    197452013      0.264664
4    241042013      0.036268


In [66]:
# Step 5c: Concatenate the new DTI dataframe back to the cleaned data
df_combined = pd.concat([df_clean, dti_df["DebtToIncome"]], axis=1)
print("Business-level dataframe with Debt-to-Income added:")
print(df_combined[["Business ID", "Business State", "DebtToIncome"]].head())

print("Debt-to-Income column added (via separate dataframe + concat):")
print(df_combined.head())

Business-level dataframe with Debt-to-Income added:
   Business ID Business State  DebtToIncome
0     41872013       Kentucky      0.123500
1     76232013           Iowa      0.182665
2    160992013          Texas      0.049974
3    197452013       Delaware      0.264664
4    241042013       Illinois      0.036268
Debt-to-Income column added (via separate dataframe + concat):
   Business ID Business State  Total Long-term Debt  Total Equity  \
0     41872013       Kentucky              16889000      18046000   
1     76232013           Iowa               6252000      18293621   
2    160992013          Texas              19200000     177858000   
3    197452013       Delaware             117592000     278773000   
4    241042013       Illinois               4408000      52064000   

   Debt to Equity  Total Liabilities  Total Revenue  Profit Margin  \
0        0.935886           25986000      136753000       0.023663   
1        0.341758           14474000       34226553       0.265015

In [60]:
# Step 6: Filter rows with negative Debt-to_Equity values
neg_dte = df_combined[df_combined["Debt to Equity"] < 0]
print("Businesses with NEGATIVE Debt-to-Equity ratios:")
print(neg_dte)

Businesses with NEGATIVE Debt-to-Equity ratios:
     Business ID Business State  Total Long-term Debt  Total Equity  \
18     934562013           Ohio             263880000    -111297000   
57    8343652013     Washington              10603000     -13271658   
87    9323722013     California              21560000     -15691000   
109  10919832013           Utah               2010000      -3602481   
117  11245242013     California                556000      -2063203   
142  14535932013        Montana              16459000      -3842372   
143  14639722013       New York                187000     -13037879   

     Debt to Equity  Total Liabilities  Total Revenue  Profit Margin  \
18        -2.370953          592174000      719783000       0.320697   
57        -0.798921           16625000        8949401       0.448119   
87        -1.374036           30048000       37782000       0.505955   
109       -0.557949            6302000       17757388       0.732562   
117       -0.269484    

In [61]:
# Step 7: Group by State and calculate descriptive statistics 
metrics_cols = ["Total Long-term Debt",
                "Total Equity",
                "Total Liabilities",
                "Total Revenue",
                "Profit Margin",
                "DebtToIncome"]
                    
                
agg_results = df_combined.groupby("Business State")[metrics_cols].agg(['mean','median','min','max'])
print("State-level descriptive statistics:")
print(agg_results)

State-level descriptive statistics:
                Total Long-term Debt                                        \
                                mean        median         min         max   
Business State                                                               
Alabama                 6.743890e+08  6.743890e+08     5314000  1343464000   
Arizona                 6.508800e+07  6.508800e+07    65088000    65088000   
Arkansas                8.960450e+07  8.960450e+07    70366000   108843000   
California              3.273964e+07  4.721500e+06       15000   334997000   
Colorado                8.470025e+07  2.932700e+07     3375000   375322000   
Connecticut             1.851200e+07  1.851200e+07    18512000    18512000   
Delaware                6.156325e+07  6.016500e+07     8331000   117592000   
Florida                 1.230350e+07  1.809000e+06        3000    45593000   
Hawaii                  6.055000e+08  6.055000e+08   605500000   605500000   
Idaho                   2.46

In [64]:
# Step 8: Save all 3 files to CSV

# Businesses with negative Debt-to-Equity
neg_dte.to_csv("negative_debt_to_equity_businesses.csv", index=False)

# Business-level dataframe with Debt-to-Income
df_combined.to_csv("business_level_with_DTI.csv", index=False)

# tate-level descriptive statistics
agg_results.to_csv("state_descriptive_stats.csv")
print("Analysis complete.")

Analysis complete.
